In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
browser_path = 'C:/Users/Phons/driver/chromedriver.exe' # change this to where you installed your chromedriver
# setting the chromedriver path and initializing driver
driver = webdriver.Chrome(executable_path=browser_path)
# create master df to append to
master_df = pd.DataFrame()

In [ ]:
from time import sleep
import random
def sleep_for(opt1, opt2):
    time_for = random.uniform(opt1, opt2)
    time_for_int = int(round(time_for))
    sleep(abs(time_for_int - time_for))
    for i in range(time_for_int, 0, -1):
        sleep(1)

In [ ]:
from progressbar import ProgressBar
pbar = ProgressBar()
urls = ['https://twitter.com/KeroKeroBonito']
# how many times should the browser scroll down
scroll_down_num = 2
# the element we are obtaining from the webpage
post_element_xpath = '//div/div/article/div/div'
# loop through your list of urls
for url in pbar(urls):
    driver.get(url)
    sleep_for(10, 15)  # sleep a while
    # scroll x number of times
    for i in range(0, scroll_down_num):
        # scroll down
        sleep_for(4, 7)
    # get a list of tweets
        post_list = driver.find_elements_by_xpath(post_element_xpath)
    # get the text only from each element
        post_text = [x.text for x in post_list]
    # create temporary dataset of each tweet
        temp_df = pd.DataFrame(post_text, columns={'all_text'})
    # append the temporary dataset to the dataset we will save
        master_df = master_df.append(temp_df) 
        driver.find_element_by_xpath('//body').send_keys(Keys.END)


In [ ]:
def parse_text(text):
    # split by new line
    text_list = str.splitlines(text) 
    # get the username (always the first list element)
    username = text_list[0]  
    #print(username)
    # within the first few elements, find the element
    # with the @ symbol, this will be the user handle
    handle = ''.join(x for x in text_list[1:3] if '@' in x)
    # get the date, using the single dot to identify its 
    # index location
    dot_position = text_list[1:4].index('·')  
    date = text_list[dot_position + 2]  # date comes after dot
    # check if its a reply to someone else
    if text_list[4] == "Replying to ":
        reply_to = True
        reply_to_handle = text_list[5]
        text = text_list[6]
    else:
        reply_to = False
        reply_to_handle = ''
        # find the longest string within list index 4:6
        # this will be the tweet text
        text = (text_list[4:-3])
        text = ''.join(text)
    # return the variables we have parse from the text
    return pd.Series([username, handle,
                      date, reply_to, reply_to_handle, text])
# run the parse function via pandas apply
master_df[['username', 'handle', 'date', 'reply_to', 'reply_to_handle', 'atext']
   ] = master_df['all_text'].apply(parse_text)
# export csv
#master_df.to_csv('output.csv')

In [ ]:
#Function that removes the quoted tweet when retweeting

def no_rt(df):
    if df.find("Quote Tweet") != -1:
        temp = df.find("Quote Tweet")
        item = df[0:temp]
        return item
    else:
        return df
master_df['atext'] = master_df['atext'].apply(no_rt)

In [ ]:
#Function to remove the date added when you retweet an old tweet
import re
def rt_only(df):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    temp = df
    x = [re.search(re.escape(word) + ' [0-9]', df) for word in months]
    new = None
    for item in x:
        if item != None:
            new = item
    if new != None:
        #print(new)
        #print(temp[5])
        y = re.search("[0-9]", temp[5])
        #print(y)
        if y != None:
            #print(temp[6:])
            return temp[6:]
        else:
            #rint(temp[5:])
            return temp[5:]
    return df

master_df['atext'] = master_df['atext'].apply(rt_only)

In [ ]:
master_df

In [ ]:
for item in master_df['atext']:
    print()
    print(item)
        